In [1]:
import time
import json
from queue import Queue
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as BSoup
import urllib.request
from urllib.parse import urlparse, quote_plus
from urllib.error import URLError, HTTPError
import shutil
import re
import csv
import os
import random
import sys
import requests
import pandas as pd


In [2]:
import datetime
from dateutil.parser import isoparse

In [3]:
keywords=['먹방', 'Mukbang']

outfile='vid.csv'
count=0
scroll_times=10
api_keys=['#######################################',    #채진
          '#######################################',    #정엽1
          '#######################################',    #정엽2
          '#######################################',    #재준
          '#######################################',    #수호
          '#######################################']    #지현

startcount=600

vids=set()
channels=dict()

#out_data=pd.DataFrame([],columns=['video_id','publish_time','publish_date','channel_id','title','views','period_day','channel_title', 'subscriber_count','tags','category_id','likes','dislikes','comment_count','description','desc_len','len_title','No_tags','video_error_or_removed','trending_date','comments_disabled','ratings_disabled'])
#index=0
#next_query_token=""

In [4]:
out_data=pd.read_csv('vid.csv',index_col=0)
print(out_data.last_valid_index())
out_data.drop_duplicates(subset=['video_id'], inplace=True)
out_data.to_csv('vid.csv')
index=out_data.last_valid_index()
print(index)
with open('next_query.txt','r') as f:
    next_query_token=f.read()
    print(next_query_token)

for i, data in out_data.iterrows():
    vids.add(data['video_id'])
rel_list=list(vids)
print(out_data)
print(next_query_token)
print(index)

13262
13262
CDIQAA
          video_id publish_time publish_date                channel_id  \
0      ZBTPLVePE7M     12:00:32   2023-05-04  UCxtLc0Jqq3SKBWlIXM_OC9g   
1      aRM3hYrs8vc     08:45:01   2023-05-05  UCq9MDndXc1y3he3uPZLPskQ   
2      g7wcHPE8py4     12:15:08   2023-05-05  UCPVENfTGbgD-CUuvrN7rRGQ   
3      HmphRR9IdOk     12:00:11   2023-05-02  UCfpaSruWW3S4dibonKXENjA   
4      nUPm-loO4FE     12:00:40   2023-05-04  UCfpaSruWW3S4dibonKXENjA   
...            ...          ...          ...                       ...   
13258  uIBXibRfy6Y     09:30:08   2022-04-25  UC3fc3077IKXwM1C0rD6-uAA   
13259  dD0mk2_lvXI     05:23:15   2023-04-24  UC2-XyrN367yxKbGiPXRXBPg   
13260  VL52mhOyRLA     12:00:20   2023-05-09  UCDdPJHmxHCJLH1sDyytCATw   
13261  HimdQI73Vr0     01:04:23   2023-04-30  UC33NOYDQxLu2oYMtf3N6AAg   
13262  -wk-sGhed8U     03:37:36   2023-04-17  UCXVHXiWhW2pgSsR-yoe2R-g   

                                                   title    views  period_day  \
0      ASMR

In [5]:
#video_id	published_at	channel_id	video_title	view_count	period_day	channel_title	subscriber_count

#cats = ['trending_date','comments_disabled','ratings_disabled','','','']

thumbnail_types=['default','medium','high','standard']
#rel_list=list(vids)

for api_key in api_keys:
    query_URL="https://www.googleapis.com/youtube/v3/videos?key="+api_key+"&part=snippet&part=statistics&part=topicDetails&id="
    channel_query_URL="https://youtube.googleapis.com/youtube/v3/channels?part=statistics&key="+api_key+"&id="
    search_query_URL="https://youtube.googleapis.com/youtube/v3/search?part=snippet&videoDuration=medium&maxResults=50&type=video&key="+api_key+"&q=Mukang"
    search_rel_URL="https://youtube.googleapis.com/youtube/v3/search?part=snippet&videoDuration=medium&maxResults=50&type=video&key="+api_key+"&relatedToVideoId="
    

    for rel_index in range(startcount,startcount+60):
        rel_vid=rel_list[rel_index]
        print("\n"+str(rel_index)+", Relative to ", rel_vid)
        url=search_rel_URL+rel_vid
        try: 
            search_r=requests.get(url=url)
            response=search_r.json()

            items=response['items']

            for item in items:
                vid=item['id']['videoId']
                if vid not in vids:
                    vids.add(vid)
                    req_url=query_URL+vid
                    try:
                        r = requests.get(url = req_url)
                        response=r.json()
                        item=response['items'][0]
                        is_mukbang=False
                        try: 
                            topics=item['topicDetails']
                            for topic in topics['topicCategories']:
                                if "Food" in topic:
                                    is_mukbang=True
                                    break
                            if not is_mukbang:
                                continue
                        except KeyError:
                            pass
                        
                        out_data.loc[index,'video_id']=vid
                        out_data.loc[index,'video_error_or_removed']=False
                        published_at=isoparse(item['snippet']['publishedAt']).replace(tzinfo=None)
                        out_data.loc[index,'publish_time']=str(published_at.time())
                        out_data.loc[index, 'publish_date']=str(published_at.date())
                        channel_id=item['snippet']['channelId']
                        out_data.loc[index,'channel_id']=channel_id
                        out_data.loc[index,'title']=item['snippet']['title']
                        out_data.loc[index,'len_title']=len(str(item['snippet']['title']))
                        out_data.loc[index,'views']=item['statistics']['viewCount']
                        out_data.loc[index,'period_day']=(datetime.datetime.now()-published_at).days
                        out_data.loc[index,'channel_title']=item['snippet']['channelTitle']
                        out_data.loc[index,'description']=item['snippet']['description']
                        out_data.loc[index,'desc_len']=len(str(item['snippet']['description']))
                        tags='\"'
                        tags_num=0
                        try:
                            for t in item['snippet']['tags']:
                                tags=tags+t+','
                                tags_num+=1
                            tags=tags[0:-2]+'\"'
                        except KeyError:
                            pass
                        if tags_num==0:
                            out_data.loc[index,'No_tags']=1
                        else:
                            out_data.loc[index,'No_tags']=0
                        out_data.loc[index,'tags']=tags
                        out_data.loc[index,'category_id']=item['snippet']['categoryId']
                        try:
                            out_data.loc[index,'likes']=item['statistics']['likeCount']
                        except KeyError:
                            out_data.loc[index,'likes']=-1
                        out_data.loc[index,'dislikes']=False
                        try:
                            out_data.loc[index,'comment_count']=item['statistics']['commentCount']
                        except KeyError:
                            out_data.loc[index,'comment_count']=-1

                        #cant retrieve this data
                        out_data.loc[index, 'trending_date']=0
                        out_data.loc[index, 'comments_disabled']=False
                        out_data.loc[index, 'ratings_disabled']=False

                        if channel_id not in channels:
                            c_req_url=channel_query_URL+channel_id
                            try:
                                channel_r=requests.get(url=c_req_url)
                                c_response=channel_r.json()
                                subscribers=c_response['items'][0]['statistics']['subscriberCount']
                                channels[channel_id]=subscribers
                                out_data.loc[index, 'subscriber_count']=subscribers
                            except requests.exceptions.RequestException as erra:
                                out_data.to_csv('vid.csv')
                                print("Exception : ", erra)
                                break
                        else:
                            out_data.loc[index, 'subscriber_count']=channels[channel_id]

                        for types in thumbnail_types:
                            try:
                                thumbnail_url=item['snippet']['thumbnails'][types]['url']
                                img_data = requests.get(thumbnail_url).content
                                with open(os.path.join('thumbnails',types,types+'_'+vid+'.jpg'), 'wb') as handler:
                                    handler.write(img_data)
                            except KeyError:
                                pass
                        index+=1
                        print("\t",index, end="")
                    except requests.exceptions.RequestException as erra:
                        out_data.to_csv('vid.csv')
                        print("Exception : ", erra)
                        break
                
                else:
                    continue

        except requests.exceptions.RequestException as erra:
            out_data.to_csv('vid.csv')
            print("Exception : ", erra)

    out_data.to_csv("vid.csv")
    startcount+=60


300, Relative to  XDP6oXN6TMw
	 7661	 7662	 7663	 7664	 7665	 7666	 7667	 7668	 7669	 7670	 7671	 7672	 7673	 7674	 7675	 7676	 7677	 7678	 7679	 7680	 7681	 7682	 7683	 7684
301, Relative to  IIN0-AxTaXk
	 7685	 7686	 7687	 7688	 7689	 7690	 7691	 7692	 7693	 7694	 7695	 7696	 7697	 7698	 7699	 7700	 7701	 7702	 7703	 7704	 7705	 7706	 7707	 7708	 7709	 7710	 7711	 7712	 7713	 7714	 7715
302, Relative to  G_ypvdzzJzA
	 7716	 7717	 7718	 7719	 7720	 7721	 7722	 7723	 7724	 7725	 7726	 7727	 7728	 7729	 7730	 7731	 7732	 7733	 7734	 7735	 7736	 7737	 7738	 7739	 7740	 7741	 7742	 7743	 7744
303, Relative to  V3dEnICJvtc
	 7745	 7746	 7747	 7748	 7749	 7750	 7751	 7752
304, Relative to  JxiOEvMomy4
	 7753	 7754	 7755	 7756	 7757	 7758	 7759	 7760	 7761	 7762	 7763	 7764	 7765	 7766	 7767	 7768	 7769	 7770	 7771	 7772	 7773	 7774	 7775	 7776	 7777	 7778	 7779	 7780	 7781	 7782
305, Relative to  NXg84Ufn8Xw
	 7783	 7784	 7785	 7786	 7787	 7788	 7789	 7790	 7791	 7792	 7793	 7794	 7795	 77

KeyError: 'items'

In [6]:
print(rel_index)
out_data.to_csv('vid.csv')
print(out_data.last_valid_index())


NameError: name 'rel_index' is not defined

In [82]:
#NO
'''
#video_id	published_at	channel_id	video_title	view_count	period_day	channel_title	subscriber_count

#cats = ['trending_date','comments_disabled','ratings_disabled','','','']

thumbnail_types=['default','medium','high','standard']

for scroll in range(scroll_times+1):
    print("\nScroll ", scroll)
    url=search_query_URL
    if scroll!=0:
        if next_query_token!="":
            url=search_query_URL+"&pageToken="+next_query_token
        else:
            break
    try: 
        search_r=requests.get(url=url)
        response=search_r.json()

        items=response['items']
        try:
            next_query_token=response['nextPageToken']
            with open('next_query.txt','w') as f:
                f.write(next_query_token)
        except KeyError:
            next_query_token=""
            with open('next_query.txt','w') as f:
                f.write(next_query_token)

        for item in items:
            vid=item['id']['videoId']
            if vid not in vids:
                vids.add(vid)
                req_url=query_URL+vid
                try:
                    r = requests.get(url = req_url)
                    response=r.json()
                    item=response['items'][0]
                    is_mukbang=False
                    try: 
                        topics=item['topicDetails']
                        for topic in topics['topicCategories']:
                            if "Food" in topic:
                                is_mukbang=True
                                break
                        if not is_mukbang:
                            continue
                    except KeyError:
                        pass

                    out_data.loc[index,'video_id']=vid
                    out_data.loc[index,'video_error_or_removed']=False
                    published_at=isoparse(item['snippet']['publishedAt']).replace(tzinfo=None)
                    out_data.loc[index,'publish_time']=str(published_at.time())
                    out_data.loc[index, 'publish_date']=str(published_at.date())
                    channel_id=item['snippet']['channelId']
                    out_data.loc[index,'channel_id']=channel_id
                    out_data.loc[index,'title']=item['snippet']['title']
                    out_data.loc[index,'len_title']=len(str(item['snippet']['title']))
                    out_data.loc[index,'views']=item['statistics']['viewCount']
                    out_data.loc[index,'period_day']=(datetime.datetime.now()-published_at).days
                    out_data.loc[index,'channel_title']=item['snippet']['channelTitle']
                    out_data.loc[index,'description']=item['snippet']['description']
                    out_data.loc[index,'desc_len']=len(str(item['snippet']['description']))
                    tags='\"'
                    tags_num=0
                    try:
                        for t in item['snippet']['tags']:
                            tags=tags+t+','
                            tags_num+=1
                        tags=tags[0:-2]+'\"'
                    except KeyError:
                        pass
                    if tags_num==0:
                        out_data.loc[index,'No_tags']=1
                    else:
                        out_data.loc[index,'No_tags']=0
                    out_data.loc[index,'tags']=tags
                    out_data.loc[index,'category_id']=item['snippet']['categoryId']
                    try:
                        out_data.loc[index,'likes']=item['statistics']['likeCount']
                    except KeyError:
                        out_data.loc[index,'likes']=-1
                    out_data.loc[index,'dislikes']=False
                    try:
                        out_data.loc[index,'comment_count']=item['statistics']['commentCount']
                    except KeyError:
                        out_data.loc[index,'comment_count']=-1

                    #cant retrieve this data
                    out_data.loc[index, 'trending_date']=0
                    out_data.loc[index, 'comments_disabled']=False
                    out_data.loc[index, 'ratings_disabled']=False

                    if channel_id not in channels:
                        c_req_url=channel_query_URL+channel_id
                        try:
                            channel_r=requests.get(url=c_req_url)
                            c_response=channel_r.json()
                            subscribers=c_response['items'][0]['statistics']['subscriberCount']
                            channels[channel_id]=subscribers
                            out_data.loc[index, 'subscriber_count']=subscribers
                        except requests.exceptions.RequestException as erra:
                            out_data.to_csv('vid.csv')
                            with open('next_query.txt','w') as f:
                                f.write(next_query_token)
                            print("Exception : ", erra)
                            break
                    else:
                        out_data.loc[index, 'subscriber_count']=channels[channel_id]

                    for types in thumbnail_types:
                        try:
                            thumbnail_url=item['snippet']['thumbnails'][types]['url']
                            img_data = requests.get(thumbnail_url).content
                            with open(os.path.join('thumbnails',types,types+'_'+vid+'.jpg'), 'wb') as handler:
                                handler.write(img_data)
                        except KeyError:
                            pass
                    index+=1
                    print("\t",index, end="")
                except requests.exceptions.RequestException as erra:
                    out_data.to_csv('vid.csv')
                    print("Exception : ", erra)
                    break
            
            else:
                continue

    except requests.exceptions.RequestException as erra:
        out_data.to_csv('vid.csv')
        print("Exception : ", erra)

out_data.to_csv("vid.csv")
'''

'\n#video_id\tpublished_at\tchannel_id\tvideo_title\tview_count\tperiod_day\tchannel_title\tsubscriber_count\n\n#cats = [\'trending_date\',\'comments_disabled\',\'ratings_disabled\',\'\',\'\',\'\']\n\nthumbnail_types=[\'default\',\'medium\',\'high\',\'standard\']\n\nfor scroll in range(scroll_times+1):\n    print("\nScroll ", scroll)\n    url=search_query_URL\n    if scroll!=0:\n        if next_query_token!="":\n            url=search_query_URL+"&pageToken="+next_query_token\n        else:\n            break\n    try: \n        search_r=requests.get(url=url)\n        response=search_r.json()\n\n        items=response[\'items\']\n        try:\n            next_query_token=response[\'nextPageToken\']\n            with open(\'next_query.txt\',\'w\') as f:\n                f.write(next_query_token)\n        except KeyError:\n            next_query_token=""\n            with open(\'next_query.txt\',\'w\') as f:\n                f.write(next_query_token)\n\n        for item in items:\n      

In [83]:
'''
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("enable-automation")
#chrome_options.add_argument("--disable-dev-shm-usage")
#chrome_options.add_argument("--disable-browser-side-navigation")
#chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--log-level=3')
'''

'\nchrome_options=webdriver.ChromeOptions()\nchrome_options.add_argument("start-maximized")\nchrome_options.add_argument("enable-automation")\n#chrome_options.add_argument("--disable-dev-shm-usage")\n#chrome_options.add_argument("--disable-browser-side-navigation")\n#chrome_options.add_argument(\'--disable-infobars\')\nchrome_options.add_argument(\'--no-sandbox\')\n#chrome_options.add_argument(\'--headless\')\n#chrome_options.add_argument(\'--disable-gpu\')\nchrome_options.add_argument(\'--log-level=3\')\n'

In [84]:
# caps=DesiredCapabilities.CHROME
# caps['goog:loggingPrefs']={'performance':'ALL'}

# page_load_timeout=60
# default_timeout=5

# driver=webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),desired_capabilities=caps,options=chrome_options)
# driver.set_page_load_timeout(page_load_timeout)

In [85]:
"""
for keys in range(len(keywords)):
    keyword=keywords[keys]
    print(keyword)
    try:
        driver.get("https://www.youtube.com/results?search_query="+keyword)
        time.sleep(default_timeout)
        for i in range(scroll_times):
            print("Scroll: ",i,end="")
            BS_obj=BSoup(driver.page_source, "html.parser")
            contents=BS_obj.find_all('a',{'id':'thumbnail', 'href':True})
            print("\tlen", len(contents))
            if len(contents)==0:
                break
            for content in contents:
                count+=1
                href=content['href']
                #/watch?v=O7QK7S0tIRM&pp=ygUG66i567Cp
                if 'short' in href:
                    continue
                if '&' in href:
                    href=href.replace('&','=')
                vid=href.split('=')[1]
                vids.add(vid)
            #print("\tCRAWL")
                
            #move to end of search
            continue_element=driver.find_element(By.TAG_NAME, 'ytd-continuation-item-renderer')
            action=ActionChains(driver)
            action.move_to_element(continue_element).perform()
            #print("\tSCROLL")
            time.sleep(default_timeout)

            #delete previous search list
            while True:                
                lists=driver.find_elements(By.TAG_NAME, 'ytd-item-section-renderer')
                if len(lists)==2:
                    break
                else:
                    time.sleep(1)
            driver.execute_script("document.getElementsByTagName('ytd-item-section-renderer')[0].remove()")
            #print("\tREMOVE PREVIOUS")
            
            
            continues=driver.find_elements(By.TAG_NAME, 'ytd-continuation-item-renderer')
            if len(continues)==2:
                driver.execute_script("document.getElementsByTagName('ytd-continuation-item-renderer')[0].remove()")
                time.sleep(1)
                    
    #        page=BS_obj.encode_contents().decode("utf-8","ignore")
        
        
    except BaseException as ex:
        print(ex)
        pass
    
driver.close()
"""
#LUtXOyzgRXM

'\nfor keys in range(len(keywords)):\n    keyword=keywords[keys]\n    print(keyword)\n    try:\n        driver.get("https://www.youtube.com/results?search_query="+keyword)\n        time.sleep(default_timeout)\n        for i in range(scroll_times):\n            print("Scroll: ",i,end="")\n            BS_obj=BSoup(driver.page_source, "html.parser")\n            contents=BS_obj.find_all(\'a\',{\'id\':\'thumbnail\', \'href\':True})\n            print("\tlen", len(contents))\n            if len(contents)==0:\n                break\n            for content in contents:\n                count+=1\n                href=content[\'href\']\n                #/watch?v=O7QK7S0tIRM&pp=ygUG66i567Cp\n                if \'short\' in href:\n                    continue\n                if \'&\' in href:\n                    href=href.replace(\'&\',\'=\')\n                vid=href.split(\'=\')[1]\n                vids.add(vid)\n            #print("\tCRAWL")\n                \n            #move to end of se